In [2]:
from astropy.table import Table
from pathlib import Path
from eazy import filters
from eazy.utils import path_to_eazy_data
import numpy as np


WD = Path.cwd()
FILTERDIR_7DT = WD / "data" / "filters" / "7DT_filters_v2"
FILTERDIR_SPHEREX= WD / "data" / "filters" / "fiducial_filters_102"
filtpaths = list(FILTERDIR_7DT.glob("*.csv"))
filtpaths.sort()
filtpaths

[PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/4000_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/4250_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/4500_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/4750_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/5000_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/5250_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/5500_fitted.csv'),
 PosixPath('/data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/data/filters/7DT_filters_v2/5750_fitted.csv'),
 PosixPath('/data1/hbahk/spherex

Reduce filter with RDP algorithm.

In [5]:
def reduce_filter(filt, epsilon_rdp=1e-5):
    """
    Reduce the number of points in the filter curve using the Ramer-Douglas-Peucker
    (RDP) algorithm. The filter curve is first smoothed using a median filter, then the
    pivot wavelength is calculated. The curve is then clipped around the pivot
    wavelength and the RDP algorithm is applied to the clipped curve.

    Parameters:
    filt (FilterDefinition): The filter curve to be reduced.
    epsilon_rdp (float, optional): The epsilon value for the RDP algorithm.
        Default is 1e-5.

    Returns:
    wave_rdp (array-like): The reduced wavelength array.
    thru_rdp (array-like): The reduced throughput array.
    """
    from scipy import signal
    from rdp import rdp

    thru_med = signal.medfilt(filt.throughput, kernel_size=5)

    pivot = np.sqrt(
        np.trapz(thru_med * filt.wave, filt.wave)
        / np.trapz(thru_med / filt.wave, filt.wave)
    )

    clip_mask = (filt.wave > pivot - 3 * filt.rectwidth) & (
        filt.wave < pivot + 3 * filt.rectwidth
    )
    wave_clip, thru_med_clip = filt.wave[clip_mask], thru_med[clip_mask]

    pivot_clip = np.sqrt(
        np.trapz(thru_med_clip * wave_clip, wave_clip)
        / np.trapz(thru_med_clip / wave_clip, wave_clip)
    )

    clip_mask_2nd = (filt.wave > pivot_clip - 1.5 * filt.rectwidth) & (
        filt.wave < pivot_clip + 1.5 * filt.rectwidth
    )
    wave_clip_2nd, thru_med_clip_2nd = filt.wave[clip_mask_2nd], thru_med[clip_mask_2nd]

    # Ramer-Douglas-Peucker algorithm.
    rdp_reduced = rdp(
        np.array([wave_clip_2nd, thru_med_clip_2nd]).T, epsilon=epsilon_rdp
    )
    wave_rdp, thru_rdp = rdp_reduced.T

    return wave_rdp, thru_rdp

In [3]:
import os
def make_eazy_filters_spherex(filtdir, Nchan=17, Ndet=6, epsilon_rdp=1e-5,
                              path_default_filter=os.path.join(path_to_eazy_data(), 'FILTER.RES.latest')):
    
    latest_filters = filters.FilterFile(path_default_filter)
    tempfile = Path('tempfile')
    with open(tempfile, 'w') as f:
        for j in range(Ndet):
            for i in range(Nchan):
                filt = Table.read(filtdir/f'filt_det{j+1}_{i+1}.csv',
                                  format='ascii.no_header',
                                  names=['wave', 'throughput'])
                res = filters.FilterDefinition(wave=filt['wave'],
                                               throughput=filt['throughput'],
                                               name=f'SPHEREx_Band{j+1}_{i+1}')
                wred, tred = reduce_filter(res, epsilon_rdp)
                
                res = filters.FilterDefinition(wave=wred, throughput=tred,
                                               name=f'SPHEREx_Band{j+1}_{i+1}')
                f.write(res.for_filter_file() + '\n')

    temp_filters = filters.FilterFile('tempfile')
    tempfile.unlink()
    
    for filt in temp_filters.filters:
        latest_filters.filters.append(filt)
        
    return latest_filters

In [7]:
path_default_filter = Path(path_to_eazy_data()) / "FILTER.RES.latest"

path_output_filter = WD / "FILTER.RES+SPHEREx+7DT_reduced.res"
epsilon_rdp = 1e-5

# latest_filters = filters.FilterFile(path_default_filter)
latest_filters = make_eazy_filters_spherex(FILTERDIR_SPHEREX,
                                           Nchan=17, Ndet=6, epsilon_rdp=epsilon_rdp,
                                           path_default_filter=path_default_filter)
tempfile = Path("tempfile")
with open(tempfile, "w") as f:
    for filtpath in filtpaths:
        filt = Table.read(
            filtpath, format="ascii.csv", names=["id", "wave", "throughput"]
        )
        res = filters.FilterDefinition(
            wave=filt["wave"] * 10, # Convert to Angstrom from nm
            throughput=filt["throughput"],
            name=f"7DT_{filtpath.stem}",
        )
        
        wred, tred = reduce_filter(res, epsilon_rdp)
                
        res = filters.FilterDefinition(wave=wred, throughput=tred, name=f"7DT_{filtpath.stem}")
        f.write(res.for_filter_file() + "\n")

temp_filters = filters.FilterFile("tempfile")
tempfile.unlink()

for filt in temp_filters.filters:
    latest_filters.filters.append(filt)

latest_filters.write(path_output_filter.as_posix())

Wrote </data1/hbahk/spherex-photoz/spherex-challenge/joint_challenge/FILTER.RES+SPHEREx+7DT_reduced.res[.info]>
